In [68]:
from __future__ import print_function
import glob
from tqdm.auto import tqdm
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import SimpleITK as sitk
import nrrd
from bs4 import BeautifulSoup
import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import six
import pickle

## 1. Image preprocessing

### 1.1 Fill the tumor region

We first fill the whole tumor region in MRI images.

In [3]:
def getMask(srcpath):
    """
    Find each tumor and fille the tumor by 1.
    """
    imgCV = cv2.imread(srcpath)
    img = cv2.cvtColor(imgCV, cv2.COLOR_BGR2RGB)
    
    text1_in_image = [255, 128,64]
    text2_in_image = [128, 128,255]

    H,W,C = img.shape

    mask = np.where( img[:,:,0]<60, img[:,:,1]>100, img[:,:,2]<60)
    mask = 1* np.array(mask).astype('uint8')
    mask_filled = np.copy(mask)
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)  
    # Todo: Check if there exists at leats one contour.
    if len(contours)>0:
        for contour in contours:
            cv2.fillPoly(mask_filled,[contour],1)
        return True, mask_filled-mask
    else:
#         print("Not found any tumor in %s" % srcpath)
        return False,mask_filled-mask

In [62]:
dataPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/"
cleanedData = dataPath+"cleanedData/"
orginalData = dataPath+"orginalData/"
maskPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/mask/"
htmlsPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/DICOM files radiomics/"
dataNrrdPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/data/"
maskNrrdPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/mask/"

Find all patients

In [5]:
patientsPaths = [v for v in glob.glob(cleanedData + "*/")]

Define which MRI images we would use

In [6]:
PreDefienedMRIFolders = ["T2U","T2M"]

In [8]:
lostDataPaths = []
notLabelledImages = []
for i in tqdm(list(range(len(patientsPaths))), "Patient"):
    pat = patientsPaths[i].split("\\")[-2]
    MRIsPaths = [v for v in glob.glob(patientsPaths[i] + "*/")]
#     print(MRIsPaths)
    # if exist T2M
    T2M_Path = [v for v in MRIsPaths if v.split("\\")[-2].find("T2M")>0 and v.split("\\")[-2].find("T2M+")==-1 and v.split("\\")[-2].find("frisk")==-1  ]
    if len(T2M_Path)<1:
        print("We cannot find T2M path in ", pat)
        continue
        
    T2U_Path = [v for v in MRIsPaths if v.split("\\")[-2].find("T2U")>0]
    if len(T2U_Path)<1:
        print("We cannot find T2U path in ", pat)
        continue
    images = [v for v in glob.glob(T2U_Path[0] + "*.tiff")]
    if len(images)==0:
        lostDataPaths.append(T2U_Path[0])
        continue    

    images = [v for v in glob.glob(T2M_Path[0] + "*.tiff")]
    if len(images)==0:
        lostDataPaths.append(T2M_Path[0])
        continue
#     print(T2U_Path[0].replace(dataPath,maskPath))

    os.makedirs(T2U_Path[0].replace(dataPath,maskPath), exist_ok=True)
    os.makedirs(T2M_Path[0].replace(dataPath,maskPath), exist_ok=True)
    atLeastOneLabel = False
    for im in images:
        islabelled, maski = getMask(srcpath=im)
        if not atLeastOneLabel:
            if islabelled:
                atLeastOneLabel = True
#         print(T2U_Path[0].replace(dataPath,maskPath), im.replace(dataPath,maskPath))
        cv2.imwrite(im.replace(dataPath,maskPath).replace('T2M', 'T2U'), maski)
        cv2.imwrite(im.replace(dataPath,maskPath), maski)
    if not atLeastOneLabel:
        notLabelledImages.append(im.split("\\")[-2])
#         print()

Patient:   0%|          | 0/65 [00:00<?, ?it/s]

Lost imgae data in

In [16]:
lostDataPaths

['C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/cleanedData\\Pat100\\Pat100T2M\\',
 'C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/cleanedData\\Pat104\\Pat104T2U\\',
 'C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/cleanedData\\Pat106\\Pat106T2M\\',
 'C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/cleanedData\\Pat68\\Pat68T2M\\',
 'C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/data/cleanedData\\Pat70\\Pat70T2M\\']

### 1.2 Extract basic information from html files

In [21]:
patientsHtmlPaths = [v for v in glob.glob(htmlsPath + "*.html")]

In [22]:
def findout_id(url):
    f = open(url,mode='r', encoding='UTF-16LE')
    soup = BeautifulSoup(f, features="lxml")
    table = soup.find('table', class_='dataTable')
    name = table.find("td", text="Patients Name").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    stage = table.find("td", text="Tamar Scan Params").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    slice_thickness = table.find("td", text="Slice Thickness").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    pixek_spacing = table.find("td", text="Pixel Spacing").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    spacing_between_slices = table.find("td", text="Spacing Between Slices").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    patients_sex = table.find("td", text="Patients Sex").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
    patients_weight = table.find("td", text="Patients Weight").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
#     creation_date = table.find("td", text="Instance Creation Date").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
#     birthday = table.find("td", text="Patients Birth Date").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text
#     age = (int(creation_date)-int(birthday))/10000
    pixek_spacing_0, pixek_spacing_1 = pixek_spacing.split("\\")
    patients_sex = '1' if patients_sex=='F' else '0'
    if stage != "T2":
        return "Not T2"
    if name.find("-")>-1:
        return [name.split("_")[-1], spacing_between_slices, pixek_spacing_0, pixek_spacing_1, slice_thickness, patients_sex, patients_weight]
    else:
        a = list(name.split("_")[-1])
        return [''.join(a[:-1]+['-']+[a[-1]]), spacing_between_slices, pixek_spacing_0, pixek_spacing_1, slice_thickness, patients_sex, patients_weight]

In [23]:
patient_id = {}
for p in patientsHtmlPaths:
    pat = p.split("\\")[-1].split("_")[0][3:].strip()
    html_result = findout_id(p)
    if html_result=="Not T2":
        continue
    patient_id[html_result[0]] = [pat]+html_result[1:]

In [24]:
print("There are totally ", len(patient_id), "patients with DICOM HTML files")

There are totally  61 patients with DICOM HTML files


Load a xlsx file

In [25]:
outcomePath2 = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/Outcome data radiomics/Radiomics_outcome_deidentified.xlsx"
df2 = pd.read_excel(outcomePath2)

In [26]:
patient_outcome = {}
res3 = []
for k,v in patient_id.items():
#     print(v)
    out = df2.loc[df2.Column1==k].iloc[:, [-1]].values
    if out is not None:
        if len(out)>1: 
#             print(k,v, out[0][0])
            if out[0][0].strip()=="Not assessable":
                continue
            patient_outcome[v[0].strip()] = [v[1],v[2],v[3], v[4], v[5], v[6], out[0][0].strip()]
#             res3.append([int(v[0].strip()), out[0][0].strip()])
        elif len(out)==1:
#             print(k,v, out[0][0])
            if out[0][0].strip()=="Not assessable":
                continue
            patient_outcome[v[0].strip()] = [v[1],v[2],v[3], v[4], v[5], v[6], out[0][0].strip()]
#             res3.append([int(v[0].strip()), out[0][0].strip()])

In [28]:
print("The number of patients having T2U and T2M MRI images AND having outcome:", len(patient_outcome))

The number of patients having T2U and T2M MRI images AND having outcome: 40


In [32]:
print("Save result to patient_outcome.npy")
np.save("./patient_outcome.npy", patient_outcome)

Save result to patient_outcome.npy


### 1.3 Convert .tiff image to 3D .nrrd image

In [33]:
output_T2_info = np.load("./patient_outcome.npy", allow_pickle=True).item()

In [34]:
def obtainMetaDataFromDICOM(id_):
    r = output_T2_info.get(str(id_))
    if not r:
        return None
    return list(map(float, r[0:4]))

In [35]:
def convertImageeNrrd(sourcePath, targetPath, id_):
    images = [v for v in glob.glob(sourcePath + "*.tiff")]
#     print(len(images))
    if len(images)>0:
        targetFileName = targetPath + targetPath.split("\\")[-2]+".nrrd"
        s = np.unique( list(map(len,  list(map(lambda path: cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY),sorted(images)  )))) )
        if len(s)>1:
            raise ValueError("Wrong image shape", sourcePath)
        imgs = np.stack( list(map(lambda path: cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY),sorted(images))) )
#         out = sitk.GetImageFromArray(imgs)
        
        MetaData = obtainMetaDataFromDICOM(id_)
        if MetaData:
#             out.SetSpacing(space)
#             sitk.WriteImage(out, targetFileName)
            header = {'spacings':MetaData[0:3],'thicknesses': [float('nan'),float('nan'),MetaData[-1]]}
            nrrd.write(targetFileName, imgs,header,index_order='C')
            return True
        else:
            return False

Generate NRRD file

In [38]:
lostDataPaths = []
c=0
for i in tqdm(list(range(len(patientsPaths))), "Patient"):
    pat = patientsPaths[i].split("\\")[-2]
#     print(pat)
    MRIsPaths = [v for v in glob.glob(patientsPaths[i] + "*/")]
#     print(MRIsPaths)
    # if exist T2M
    T2M_Path = [v for v in MRIsPaths if v.split("\\")[-2].find("T2M")>0 and v.split("\\")[-2].find("T2M+")==-1 and v.split("\\")[-2].find("frisk")==-1  ]
    if len(T2M_Path)<1:
        print("We cannot find T2M path in ", pat)
        continue
        
    T2U_Path = [v for v in MRIsPaths if v.split("\\")[-2].find("T2U")>0]
    if len(T2U_Path)<1:
        print("We cannot find T2U path in ", pat)
        continue
    images = [v for v in glob.glob(T2U_Path[0] + "*.tiff")]
    if len(images)==0:
        lostDataPaths.append(T2U_Path[0])
        continue    

    images = [v for v in glob.glob(T2M_Path[0] + "*.tiff")]
    if len(images)==0:
        lostDataPaths.append(T2M_Path[0])
        continue
    # T2U image nrrd

    src,tar = T2U_Path[0], T2U_Path[0].replace(dataPath,dataNrrdPath)
    os.makedirs(tar, exist_ok=True)
    flag = convertImageeNrrd(src,tar, int(pat[3:]))
    # T2M image nrrd
    if not flag:
        continue
        
    src,tar = T2M_Path[0], T2M_Path[0].replace(dataPath,dataNrrdPath)
    os.makedirs(tar, exist_ok=True)
    flag = convertImageeNrrd(src,tar,int(pat[3:]))
    if not flag:
        continue
    # T2M mask nrrd

    src,tar = T2M_Path[0].replace(dataPath,maskPath), T2M_Path[0].replace(dataPath,maskNrrdPath)
    os.makedirs(tar, exist_ok=True)
    flag = convertImageeNrrd(src,tar, int(pat[3:]))
    # T2U mask nrrd
    if not flag:
        continue
#     src,tar = T2U_Path[0].replace(dataPath,maskPath), T2U_Path[0].replace(dataPath,maskNrrdPath)
#     os.makedirs(tar, exist_ok=True)
#     flag = convertImageeNrrd(src,tar, int(pat[3:]))
    c = c+1
print(c)


Patient:   0%|          | 0/65 [00:00<?, ?it/s]

40


## 2. Extract Features

In [40]:
lostDataPaths = np.array(['Pat100','Pat106','Pat68','Pat104','Pat70','Pat105'], dtype='<U33')

In [41]:
paramPath = os.path.join('Params_customization.yaml')

In [43]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
Enabled filters:
	 {'Original': {}, 'LoG': {'sigma': [1.0, 1.5, 2.0, 2.5, 3.0]}}
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares']}


In [64]:
dataNrrdPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/data/cleanedData\\"
maskNrrdPath = "C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/mask/cleanedData\\"

In [65]:
patientsPaths = [v for v in glob.glob(cleanedData + "*/")]
patientMaskPaths  = [v for v in glob.glob(maskNrrdPath + "*/")]

In [69]:
resultFilePaths = []
for i in tqdm(list(range(len(patientMaskPaths))), "Patient mask"):
    pat = patientMaskPaths[i].split("\\")[-2]
    if pat in lostDataPaths:
        print("Lost patient:", pat)
        continue
#     print(patientMaskPaths[i].replace(maskNrrdPath,dataNrrdPath))
    MRIsDataPaths = [v for v in glob.glob(patientMaskPaths[i].replace(maskNrrdPath,dataNrrdPath) + "*\\*.nrrd")]
    MRIsMaskPaths = [v for v in glob.glob(patientMaskPaths[i] + "*\\*.nrrd")]
    T2M_Path = [v for v in MRIsMaskPaths if v.split("\\")[-2].find("T2M")>0 and v.split("\\")[-2].find("T2M+")==-1 and v.split("\\")[-2].find("frisk")==-1  ]
    if len(T2M_Path)<1:
        print("We cannot find T2M path in ", pat)
        continue
#     print(T2M_Path)
        
    T2U_Path = [v for v in MRIsDataPaths if v.split("\\")[-2].find("T2U")>0]
    if len(T2U_Path)<1:
        print("We cannot find T2U path in ", pat)
        continue
#     print(T2U_Path)
    image_folder_path = '/'.join(T2U_Path[0].split("\\")[0:2])+'/'
    image_path = T2U_Path[0]

    
#     mask_folder_path = T2M_Path[0]
#     mask_path = mask_folder_path+mask_folder_path.split("\\")[-2]+".nrrd"    
    mask_path = T2M_Path[0]
    result_path = image_folder_path.replace("data", "result")
    resultFilePath = result_path +image_folder_path.split("/")[-2]+".pickle"
#     print(image_path)
#     print(mask_path)
    result = extractor.execute(image_path, mask_path)
    os.makedirs(result_path, exist_ok=True)            
    pickle.dump(result, open(resultFilePath, 'wb'))
    print("process result in ", resultFilePath)
    resultFilePaths.append(resultFilePath)

np.save("./resultFilePaths.npy", np.array(resultFilePaths))

Patient mask:   0%|          | 0/40 [00:00<?, ?it/s]

process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat1/Pat1.pickle
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat102/Pat102.pickle
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat103/Pat103.pickle
Lost patient: Pat105
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat11/Pat11.pickle
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat12/Pat12.pickle
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat13/Pat13.pickle
process result in  C:/Users/nangu/OneDrive - Uppsala universitet/ComputationalScienceProjectData/nrrd/result/cleanedData/Pat14/Pat14.pick